# JAVIS Fine-tuning (Google Colab)

Qwen2.5-7B-Instruct QLoRA 파인튜닝

**중요: 셀을 순서대로 실행하세요!**

**필요사항:**
- Google Colab (무료 T4 GPU)
- HuggingFace 토큰
- 학습 데이터 (JSONL)

## 1. 환경 설정

In [ ]:
# [1] GPU 확인
!nvidia-smi

In [ ]:
# [2] 패키지 설치 (2-3분 소요)
!pip install -q torch transformers datasets peft trl bitsandbytes accelerate scipy

In [ ]:
# [3] 전체 import 및 설정
import json
import torch
from datetime import datetime
from datasets import Dataset
from google.colab import files
from huggingface_hub import login
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

# 설정값
BASE_MODEL = "Qwen/Qwen2.5-7B-Instruct"
OUTPUT_DIR = "./javis-adapter"
EPOCHS = 3
BATCH_SIZE = 2
LEARNING_RATE = 2e-4
LORA_R = 64
LORA_ALPHA = 16
MAX_SEQ_LENGTH = 1024

print("Import 완료!")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

In [ ]:
# [4] HuggingFace 로그인
# https://huggingface.co/settings/tokens 에서 토큰 발급
HF_TOKEN = "hf_xxxxxxxxxxxxxxxxxxxxx"  # @param {type:"string"}
login(token=HF_TOKEN)

## 2. 학습 데이터 업로드

In [ ]:
# [5] JSONL 파일 업로드
# 로컬에서 export한 conversations_xxx.jsonl 파일 선택
print("conversations_xxx.jsonl 파일을 선택하세요")
uploaded = files.upload()

TRAINING_DATA = list(uploaded.keys())[0]
print(f"\n업로드 완료: {TRAINING_DATA}")

In [ ]:
# [6] 데이터 로드 및 확인
def load_training_data(path):
    conversations = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                conversations.append(json.loads(line))
    return Dataset.from_list(conversations)

dataset = load_training_data(TRAINING_DATA)
print(f"로드된 대화 수: {len(dataset)}")
print(f"\n첫 번째 대화 예시:")
print(json.dumps(dataset[0], indent=2, ensure_ascii=False))

## 3. 모델 로드

In [ ]:
# [7] 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 데이터 포맷팅
def format_conversation(example):
    text = tokenizer.apply_chat_template(
        example['messages'],
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

dataset = dataset.map(format_conversation, remove_columns=dataset.column_names)
print("토크나이저 로드 완료")
print(f"\n포맷된 예시:\n{dataset[0]['text'][:300]}...")

In [ ]:
# [8] 모델 로드 (5-10분 소요)
print("모델 로딩 중... (5-10분 걸립니다)")

# 4-bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False

print("모델 로드 완료!")

In [ ]:
# [9] LoRA 설정
peft_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ]
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

trainable, total = model.get_nb_trainable_parameters()
print(f"학습 파라미터: {trainable:,} / {total:,} ({100*trainable/total:.2f}%)")

## 4. 학습 실행

In [ ]:
# [10] 학습 시작 (1-2시간 소요)
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=8,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    logging_steps=10,
    save_strategy="epoch",
    bf16=True,
    optim="paged_adamw_8bit",
    report_to="none",
    gradient_checkpointing=True,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    tokenizer=tokenizer,
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_text_field="text",
)

print("="*50)
print("학습 시작! (데이터 양에 따라 1-2시간 소요)")
print("="*50)

trainer.train()

print("\n" + "="*50)
print("학습 완료!")
print("="*50)

## 5. 저장 및 다운로드

In [ ]:
# [11] Adapter 저장
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# 메타데이터 저장
metadata = {
    "base_model": BASE_MODEL,
    "created_at": datetime.now().isoformat(),
    "training_config": {
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "learning_rate": LEARNING_RATE,
        "lora_r": LORA_R,
        "lora_alpha": LORA_ALPHA,
    },
    "dataset_size": len(dataset),
}

with open(f"{OUTPUT_DIR}/metadata.json", 'w') as f:
    json.dump(metadata, f, indent=2)

print("저장 완료!")
!ls -la {OUTPUT_DIR}

In [ ]:
# [12] ZIP으로 다운로드
!zip -r javis-adapter.zip {OUTPUT_DIR}
files.download('javis-adapter.zip')

print("\n다운로드 완료! 이 파일을 로컬에 저장하세요.")

## 6. (선택) HuggingFace 업로드

In [ ]:
# [13] HuggingFace Hub에 업로드 (선택사항)
HF_USERNAME = "your-username"  # @param {type:"string"}
REPO_NAME = "javis-adapter-v1"  # @param {type:"string"}

model.push_to_hub(f"{HF_USERNAME}/{REPO_NAME}")
tokenizer.push_to_hub(f"{HF_USERNAME}/{REPO_NAME}")

print(f"업로드 완료: https://huggingface.co/{HF_USERNAME}/{REPO_NAME}")

## 7. 테스트

In [ ]:
# [14] 파인튜닝된 모델 테스트
test_messages = [
    {"role": "user", "content": "안녕, 넌 누구야?"}
]

text = tokenizer.apply_chat_template(test_messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(text, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer.pad_token_id
)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("="*50)
print("테스트 응답:")
print("="*50)
print(response)